**BatPac** is an excel model that is used to estimate the cell cost of batteries using parameters such as material costs and chemical properties
- See here for more: https://www.anl.gov/partnerships/batpac-battery-manufacturing-cost-estimation

All of these functions are used to automate BatPac tasks, which usually require a lot of manual inputting and recording 

In [1]:
import pandas as pd
import xlwings as xw
from openpyxl import load_workbook

Automate going from changing prices in the material price spreadsheet, to those price changes being automatically implemented into the cost calculations. 

In [2]:
def price_loader(price_filepath):
    """
    Loads the material prices that will be input into batpac
    """
    price_input_location = {
        "NMC811": "B3",
        "LMO": "B4",
        "LNMO": "B5",
        "LFP": "B6",
        "lithium_metal": "B8",
        "V2O5": "B9",
        "LVO": "B10",
        "graphite": "B11",
        "silicon": "B12",
        "16%SI-64%LVO": "B13",
        "20%SI-60%LVO": "B14",
        "24%SI-56%LVO": "B15",
        "binder": "B17",
        "binder_solvant": "B18",
        "separator": "B20",
        "electrolyte": "B19"
    }
    
    price_data = pd.DataFrame({
        "material": [],
        "cost": []
    })
    
    price_file = xw.Book(price_filepath)
    
    for material, location in price_input_location.items():
        price_data.loc[len(price_data.index)] = [material, price_file.sheets["Sheet1"].range(location).value]
    return price_data

def batpac_price_loader(batpac_filepath):
    """
    Loads the prices already in batpac
    """
    batpac_material_cost_location = {
        "NMC811": ["AR149", "DF149", "DK149", "DP149", "DU149"],
        "LNMO": ["DZ149", "EE149", "EJ149", "EO149"],
        "LFP": ["BW149"],
        "LVO": ["DF155", "DZ155"],
        "16%SI":["DK155", "EE155"],
        "20%SI": ["DP155", "EJ155"],
        "24%SI": ["DU155", "EO155"],
        "graphite": ["BW155", "AR155"],
        "binder": ["AR157","AR151", "BW157", "BW151", "DF157", "DF151", "DK157", "DK151", "DP157", "DP151", "DU157", "DU151", "DZ157", "DZ151", "EE157", "EE151", "EJ157", "EJ151", "EO157", "EO151"],
        "binder_solvant": ["AR158", "AR152", "BW158", "BW152", "DF158", "DF152", "DK158", "DK152", "DP158", "DP152", "DU158", "DU152", "DZ158", "DZ152", "EE158", "EE152", "EJ158", "EJ152", "EO158", "EO152"],
        "separator": ["AR163","BW163", "DF163", "DK163", "DP163", "DU163", "DZ163", "EE163", "EJ163", "EO163"],
        "electrolyte": ["AR164","BW164", "DF164", "DK164", "DP164", "DU164", "DZ164", "EE164", "EJ164", "EO164"]
    }
        
    price_data = pd.DataFrame({
        "material": [],
        "cost": []
    })
    
    batpac = xw.Book(batpac_filepath)
    
    for material, location in batpac_material_cost_location.items():
        price_data.loc[len(price_data.index)] = [material, batpac.sheets["Chem"].range(location[0]).value]
    return price_data

def batpac_price_changer(batpac_filepath, **chem_price):
    """
    change the prices in batpac
    """
    batpac_material_cost_location = {
        "NMC811": ["AR149", "DF149", "DK149", "DP149", "DU149"],
        "LNMO": ["DZ149", "EE149", "EJ149", "EO149"],
        "LFP": ["BW149"],
        "LVO": ["DF155", "DZ155"],
        "16%SI":["DK155", "EE155"],
        "20%SI": ["DP155", "EJ155"],
        "24%SI": ["DU155", "EO155"],
        "graphite": ["BW155", "AR155"],
        "binder": ["AR157","AR151", "BW157", "BW151", "DF157", "DF151", "DK157", "DK151", "DP157", "DP151", "DU157", "DU151", "DZ157", "DZ151", "EE157", "EE151", "EJ157", "EJ151", "EO157", "EO151"],
        "binder_solvant": ["AR158", "AR152", "BW158", "BW152", "DF158", "DF152", "DK158", "DK152", "DP158", "DP152", "DU158", "DU152", "DZ158", "DZ152", "EE158", "EE152", "EJ158", "EJ152", "EO158", "EO152"],
        "separator": ["AR163","BW163", "DF163", "DK163", "DP163", "DU163", "DZ163", "EE163", "EJ163", "EO163"],
        "electrolyte": ["AR164","BW164", "DF164", "DK164", "DP164", "DU164", "DZ164", "EE164", "EJ164", "EO164"]
    }
    
    for chem in chem_price.keys():
        if chem not in batpac_material_cost_location.keys():
            raise Exception("Input not a valid chemistry") 
    
    batpac = xw.Book(batpac_filepath)
    
    for chem, new_price in chem_price.items():
        for location in batpac_material_cost_location[chem]:
            batpac.sheets["Chem"].range(location).value = new_price
    
    batpac.save()
    print("complete")
    
def cell_cost_breakdown(batpac_filepath, *args):
    """
    Gets the detailed cost breakdown from batpac
    """
    batpac = xw.Book(batpac_filepath)

    dashboard = batpac.sheets['Dashboard']
    chem_dropdown = dashboard.range('E13')

    cost_table = pd.DataFrame()
    
    chem_names = {
        "LFP-G": "LFP-G (Power)",
        "NMC-G": "NMC811-G (Power)",
        "NMC-LVO": "NMC811-LVO (Power)",
        "NMC-16%Si": "NMC811/16%Si-64%LVO (Power)",
        "NMC-20%Si": "NMC811/20%Si-60%LVO (Power)",
        "NMC-24%Si": "NMC811/24%Si-56%LVO (Power)",
        "LNMO-LVO": "LNMO-LVO (Power)",
        "LNMO-16%Si": "LNMO/16%Si-64%LVO (Power)",
        "LNMO-20%Si":"LNMO/20%Si-60%LVO (Power)",
        "LNMO-24%Si": "LNMO/24%Si-56%LVO (Power)",
    }

    for chem in args:
        
        if chem in chem_names.keys():
            current_chem = chem_names[chem]
        else:
            current_chem = chem
        
        chem_dropdown.value = current_chem

        batpac.app.calculate()
        batpac.save()

        dash_results = pd.read_excel(batpac_filepath, sheet_name = 'Dashboard', usecols = "C:D", skiprows = 196, nrows = 30)
        cell_cost_components = dash_results.drop(range(0, 3)).dropna(subset=["Battery 1"])
        nan_rowname_idxs = cell_cost_components[cell_cost_components.iloc[:,0].isnull()].index

        #add more code here if there are more rows with nonexistant names
        cell_cost_components.loc[nan_rowname_idxs, cell_cost_components.columns[0]] = "Negative electrode binder solvent (NMP), $/cell"

        rename = lambda name: name[len("Cell Materials Cost, $/cell: "):] if "Cell Materials Cost, $/cell: " in name else name
        cell_cost_components.loc[15:, cell_cost_components.columns[0]] = cell_cost_components.loc[15:, cell_cost_components.columns[0]].apply(rename)
        cell_cost_components.reset_index(drop = True, inplace = True)
        cell_cost_components.loc[2:, cell_cost_components.columns[1]] = cell_cost_components[2:][cell_cost_components.columns[1]]/batpac.sheets['Summary of Results'].range("G101").value

        cell_cost_components.rename(columns = {"Battery 1":chem}, inplace = True)
        cell_cost_components.set_index(cell_cost_components.columns[0], drop=True, inplace=True)
        cell_cost_components.index.name = None

        cell_cost_components = cell_cost_components.drop("Cell total energy, kWh")

        cost_table = pd.concat([cost_table, cell_cost_components], axis=1)

    return cost_table

def cell_breakdown_sorter(breakdown_df):
    """
    Reorders the cost breakdown into a standardized order
    """
    row_order = ['Cell cost, $/kWh',
     'Total cell-level cost, $/cell',
     'Positive electrode active material, $/cell',
     'Negative electrode active material, $/cell',
     'Negative current collector, $/cell',
     'Depreciation, $/cell',
     'Profits on initial investment, $/cell',
     'Warranty on cells, $/cell',
     'Variable overhead, $/cell',
     'Electrolyte, $/cell',
     'Research and development, $/cell',
     'Negative electrode binder solvent (NMP), $/cell',
     'Direct labor at $25/hour, $/cell',
     'General, sales, administration, $/cell',
     'Positive electrode binder solvent (NMP), $/cell',
     'Energy at $0.04/kWh, $/cell',
     'Cell container, $/unit',
     'Positive current collector, $/cell',
     'Separators, $/cell',
     'Negative terminal , $/unit',
     'Positive electrode binder, $/cell',
     'Financing, $/cell',
     'Positive electrode carbon additive, $/cell',
     'Negative electrode binder, $/cell',
     'Positive terminal, $/unit',
     'Negative electrode carbon additive, $/cell']

    ordered_breakdown = breakdown_df.loc[row_order]
    
    rename = lambda col_name: col_name[:-7] if "$/kWh" in col_name else col_name[:-8] 
    ordered_breakdown.index = ordered_breakdown.index.map(rename)
    
    return ordered_breakdown

def remove_unneeded_costs(cost_data_df, *unnecessary_costs):
    """
    Remove certain costs from the breakdown and recalculates the total cost
    """
    without_extras = cost_data_df.drop(list(unnecessary_costs))
    without_extras.loc['Total Cell Cost without unwanted ($/kWh)'] = without_extras.sum(axis = 0)
    return without_extras

def specific_cap_changer(batpac_filepath, **material_capacity):
    """
    Change the specific capacity of certain materials
    """
    batpac_material_capacity_location = {
        "NMC811": ["AR8", "DF8", "DK8", "DP8", "DU8"],
        "graphite": ["AR38", "BW38"],
        "LFP": ["BW8"]
    }
    
    batpac = xw.Book(batpac_filepath)
    chem_tab = batpac.sheets["Chem"]
    
    for material, new_cap in material_capacity.items():
        if material not in batpac_material_capacity_location.keys():
            raise Exception("Input valid material")
        
        for location in batpac_material_capacity_location[material]:
            chem_tab.range(location).value = new_cap
    
    print("Complete")
    
def performance_breakdown(batpac_filepath, *args):
    """
    Gets the detailed cost breakdown from batpac
    """
    batpac = xw.Book(batpac_filepath)

    dashboard = batpac.sheets['Dashboard']
    chem_dropdown = dashboard.range('E13')
    
    chem_names = {
        "LFP-G": "LFP-G (Power)",
        "NMC-G": "NMC811-G (Power)",
        "NMC-LVO": "NMC811-LVO (Power)",
        "NMC-16%Si": "NMC811/16%Si-64%LVO (Power)",
        "NMC-20%Si": "NMC811/20%Si-60%LVO (Power)",
        "NMC-24%Si": "NMC811/24%Si-56%LVO (Power)",
        "LNMO-LVO": "LNMO-LVO (Power)",
        "LNMO-16%Si": "LNMO/16%Si-64%LVO (Power)",
        "LNMO-20%Si":"LNMO/20%Si-60%LVO (Power)",
        "LNMO-24%Si": "LNMO/24%Si-56%LVO (Power)",
    }
    
    performance_table = pd.read_excel(batpac_filepath, sheet_name = 'Dashboard', usecols = "C", skiprows = 229, nrows = 6)
    performance_table.columns = ['Metric']
    
    for chem in args:
        if chem in chem_names.keys():
            current_chem = chem_names[chem]
        else:
            current_chem = chem
        
        chem_dropdown.value = current_chem

        batpac.app.calculate()
        batpac.save()

        chem_perf_results = pd.read_excel(batpac_filepath, sheet_name = 'Dashboard', usecols = "D", skiprows = 229, nrows = 6)
        chem_perf_results.columns = [chem]
        performance_table = pd.concat([performance_table, chem_perf_results], axis=1)
        
    return performance_table

def postitve_electrode_thickness_changer(batpac_filepath, new_thickness):
    batpac = xw.Book(batpac_filepath)
    dashboard = batpac.sheets["Dashboard"]
    dashboard.range("E18").value = new_thickness

In [ ]:
material_price_inputs = None
batpac_model = '/Users/gakuueno/Desktop/TyFast Internship/BatPac/Modeling/price_today/BatPacPriceToday.xlsm'
full_excel_output = None
excel_output_without_randd_profits_financing = '/Users/gakuueno/Desktop/TyFast Internship/BatPac/performance_results.xlsx'
sheet = 'NMC-G_loading_density'

In [ ]:
material_cost_inputs = price_loader(material_price_inputs)
material_cost_inputs

In [ ]:
batpac_price_loader(batpac_model)

In [9]:
#specific_cap_changer(batpac_model, NMC811 = 0, graphite = 0, LFP = 0) removed numbers for confidentiality

In [13]:
# new_prices = {
#      "NMC811": material_cost_inputs[material_cost_inputs['material'] == "NMC811"]['cost'].iloc[0], #try to directly reference the material_prices dataframe so the changes up there are updated here
#      "LNMO": material_cost_inputs[material_cost_inputs['material'] == "LNMO"]['cost'].iloc[0],
#      "LFP": material_cost_inputs[material_cost_inputs['material'] == "LFP"]['cost'].iloc[0],
#      "LVO": material_cost_inputs[material_cost_inputs['material'] == "LVO"]['cost'].iloc[0],
#      "16%SI": material_cost_inputs[material_cost_inputs['material'] == "16%SI-64%LVO"]['cost'].iloc[0],
#      "20%SI": material_cost_inputs[material_cost_inputs['material'] == "20%SI-60%LVO"]['cost'].iloc[0],
#      "24%SI": material_cost_inputs[material_cost_inputs['material'] == "24%SI-56%LVO"]['cost'].iloc[0],
#      "graphite": material_cost_inputs[material_cost_inputs['material'] == "graphite"]['cost'].iloc[0],
#      "binder": material_cost_inputs[material_cost_inputs['material'] == "binder"]['cost'].iloc[0],
#      "binder_solvant": material_cost_inputs[material_cost_inputs['material'] == "binder_solvant"]['cost'].iloc[0],
#      "separator": material_cost_inputs[material_cost_inputs['material'] == "separator"]['cost'].iloc[0],
#      "electrolyte": material_cost_inputs[material_cost_inputs['material'] == "electrolyte"]['cost'].iloc[0]
#     }

new_prices = {
     "binder": 0
    }

batpac_price_changer(batpac_model, **new_prices)

NameError: name 'batpac_price_changer' is not defined

In [ ]:
chem_dict = {
        "LFP-G": "LFP-G (Power)",
        "NMC-G": "NMC811-G (Power)",
        "NMC-LVO": "NMC811-LVO (Power)",
        "NMC-16%Si": "NMC811/16%Si-64%LVO (Power)",
        "NMC-20%Si": "NMC811/20%Si-60%LVO (Power)",
        "NMC-24%Si": "NMC811/24%Si-56%LVO (Power)",
        "LNMO-LVO": "LNMO-LVO (Power)",
        "LNMO-16%Si": "LNMO/16%Si-64%LVO (Power)",
        "LNMO-20%Si":"LNMO/20%Si-60%LVO (Power)",
        "LNMO-24%Si": "LNMO/24%Si-56%LVO (Power)",
        "NMC811-LTO": "NMC811-LTO",
        "LNMO-LTO": "LMO-LTO",
        "NMC811-NTO": "NMC811-NTO",
        "LNMO-NTO": "LNMO-NTO",
        "NMC811-NWO": "NMC811-NWO",
        "LNMO-NWO": "LNMO-NWO"
    }

breakdown = cell_cost_breakdown(batpac_model, *chem_dict.keys())
breakdown

In [ ]:
breakdown.to_excel(full_excel_output)

In [ ]:
sorted_breakdown = cell_breakdown_sorter(breakdown)
sorted_breakdown

In [ ]:
breakdown_without_extras = remove_unneeded_costs(sorted_breakdown, "Cell cost", "Total cell-level cost", "Profits on initial investment", "Warranty on cells")
breakdown_without_extras

In [ ]:
#breakdown_without_extras.to_excel(excel_output_without_randd_profits_financing)

book = load_workbook(excel_output_without_randd_profits_financing)

with pd.ExcelWriter(excel_output_without_randd_profits_financing, engine='openpyxl', mode='a') as writer:
    breakdown_without_extras.to_excel(writer, sheet_name=sheet, index=True)

In [ ]:
performance_excel_output = '/Users/gakuueno/Desktop/TyFast Internship/BatPac/performance_results.xlsx'
sheet = 'results'
book = load_workbook(performance_excel_output)

with pd.ExcelWriter(performance_excel_output, engine='openpyxl', mode='a') as writer:
    breakdown.to_excel(writer, sheet_name=sheet, index=True)

In [5]:
chem_dict = {
        "LFP-G": "LFP-G (Power)",
        "NMC-G": "NMC811-G (Power)",
        "NMC-LVO": "NMC811-LVO (Power)",
        "NMC-16%Si": "NMC811/16%Si-64%LVO (Power)",
        "NMC-20%Si": "NMC811/20%Si-60%LVO (Power)",
        "NMC-24%Si": "NMC811/24%Si-56%LVO (Power)",
        "LNMO-LVO": "LNMO-LVO (Power)",
        "LNMO-16%Si": "LNMO/16%Si-64%LVO (Power)",
        "LNMO-20%Si":"LNMO/20%Si-60%LVO (Power)",
        "LNMO-24%Si": "LNMO/24%Si-56%LVO (Power)",
    }

excel_output = '/Users/gakuueno/Desktop/TyFast Internship/BatPac/Constant Electrode Thickness and Loading /negative_loading_constant.xlsx'
batpac_model = '/Users/gakuueno/Desktop/TyFast Internship/BatPac/Modeling/price_today/BatPacPriceToday.xlsm'

for chem in chem_dict.keys():    
    chem_thickness = {
        "LFP-G": 0,
        "NMC-G": 0,
        "NMC-LVO": 0,
        "NMC-16%Si": 0,
        "NMC-20%Si": 0,
        "NMC-24%Si": 0,
        "LNMO-LVO": 0,
        "LNMO-16%Si": 0,
        "LNMO-20%Si": 0,
        "LNMO-24%Si": 0,
    }
    
    postitve_electrode_thickness_changer(batpac_model, chem_thickness[chem])
    
    full_cost_breakdown = cell_cost_breakdown(batpac_model, chem)
    sorted_breakdown = cell_breakdown_sorter(full_cost_breakdown)
    breakdown_without_extras = remove_unneeded_costs(sorted_breakdown, "Cell cost", "Total cell-level cost", "Profits on initial investment", "Warranty on cells")
    
    cost_sheet = chem + '_cost_breakdown'
    with pd.ExcelWriter(excel_output, engine='openpyxl', mode='a') as writer:
        breakdown_without_extras.to_excel(writer, sheet_name=cost_sheet, index=True)
    
    performance_sheet = chem + '_performance'
    performance_breakdown_data = performance_breakdown(batpac_model, chem)
    with pd.ExcelWriter(excel_output, engine='openpyxl', mode='a') as writer:
        performance_breakdown_data.to_excel(writer, sheet_name=performance_sheet, index=True)